---
Stellen Sie bitte sicher, dass alles wie vorhergesehen läuft, bevor Sie dieses Übungsblatt abgeben. **Starten Sie den Kernel neu** (in der Menüleiste die Option Kernel$\rightarrow$Restart auswählen) und **validieren** Sie anschließend das Übungsblatt (in der Menüleiste auf Validate klicken) um Rückmeldung zu eventuellen fehlenden oder fehlerhaften Eingaben zu erhalten. 

Füllen Sie alle Stellen im Übungsblatt aus, welche entweder `DEIN CODE HIER` oder "DEINE ANTWORT HIER" enthalten. Geben Sie unterhalb Ihren vollständigen Namen an.

Wenn Sie Code-Bestandteile aus anderen Quellen (wie z.B. Stackoverflow) kopieren, dann machen sie den kopierten Code in ihrer Quellcodedatei kenntlich und fügen eine Referenz auf die Quelle als Kommentar hinzu.

Wenn Sie die Aufgaben in einer Gruppe erledigen, dann fügen Sie die Namen aller Gruppenmitglieder in der nachfolgende Zelle zu `Name` und zusätzlich als Kommentar am Anfang Ihrer Quellcodedatei hinzu.

---

In [ ]:
NAME = "Gernot Luidolt"

---

<table style="width: 100%">
    <tr style="background: #ffffff">
        <td style="padding-top:25px;width: 180px"><img src="https://mci.edu/templates/mci/images/logo.svg" alt="Logo"></td>
        <td style="width: 100%">
            <div style="text-align:right; width: 100%; text-align:right"><font style="font-size:38px"><b>Methoden der Softwareentwicklung I</b></font></div>
            <div style="padding-top:0px; width: 100%; text-align:right"><font size="4"><b>WS 2023/24</b></font></div>
        </td>
    </tr>
</table>

In [ ]:
import jagl
import os

---
# Übungsblatt 5

---
Abgabe bis <b>Sonntag, 14. Jänner 2024, 23:55 Uhr</b>

---
### Achtung:

*Bitte speichern Sie Ihre Quellcode-Dateien (&ast;.c) im selben Ordner wie dieses Übungsblatt. Beachten Sie außerdem die Kommentare zur richtigen Benennung Ihrer Dateien. Nur so ist eine korrekte Abgabe und Auswertung Ihrer Lösungen möglich.*

In [ ]:
# new test suite
jagl.testsuite_begin("Exercise 1")

# delete all build artifacts
jagl.remove_paths(files=["average", "*.o"])

---
## 5.1 Durchschnitt mit dynamischer Speicherverwaltung (50%)

Implementieren Sie ein Programm, welches den Durchschnitt aller Elemente eines Integer-Arrays ausgibt. Der Benutzer soll jedes einzelne Array-Element über `stdin` eingeben. Im Gegensatz zu Aufgabe 3.3 soll der Speicher dieses Mal aber dynamisch verwaltet werden. Zu Beginn soll mit `malloc` Speicher für 5 Elemente (= Blockgröße) reserviert werden. Sobald dieser Speicher komplett befüllt ist, soll mit `realloc` Speicher für einen weiteren Block mit 5 Elementen reserviert werden. Wiederholen Sie diesen Vorgang so oft wie nötig.

Wird etwas anderes als ein Integer eingegeben, dann soll diese Eingabe ignoriert und das Einlesen fortgesetzt werden. Sobald der Benutzer eine leere Zeile eingibt, soll das Einlesen beendet werden. Dann soll das Array an eine Funktion übergeben werden, welche den Durchschnitt der Elemente berechnet und ausgibt. Dabei soll exakt eine Nachkommastelle ausgegeben werden. Am Ende soll der alloziierte Speicher explizit wieder freigegeben werden. Ein Programmablauf könnte beispielweise so aussehen:

`> Geben Sie die Elemente Ihres Arrays an:`<br/>
`> Element #1:`<br/>
5<br/>
`> Element #2:`<br/>
1<br/>
`> Element #3:`<br/>
not a number<br/>
`> Element #3:`<br/>
3<br/>
`> Element #4:`<br/>
<br/>
`> Eingabe beendet.`<br/>
`> Der Durchschnitt Ihres Arrays ist: 3.0`

In [ ]:
%%bash

# Schreiben Sie in diese Zelle den Befehl der dafür nötig ist um Ihr Programm zu kompilieren.
# Beachten Sie dabei, dass der Name ihrer Quellcode-Datei "average.c" sein sollte.
# Beachten Sie dabei, dass der Name Ihres ausführbaren Programms "average" sein sollte.

gcc -Wall -Werror -o average average.c

In [ ]:
# Weisen Sie der untenstehenden Variable den Wert True zu, sobald Sie die Aufgabe erfolgreich erledigt haben!

# Datentyp: bool
exercise_5_1_solved = True

In [ ]:
executable_name = "average"
source_files = ["average.c"]
cpp_flags = []

In [ ]:
@jagl.testcase("1", desc="Exercise Solved")
def testcase_1_1(result, suite, case):
    varname = "exercise_5_1_solved"
    result.setSucceeded()
    jagl.check_variable_exists_and_has_type(result, globals(), varname, bool)
    if result.isSucceeded():
        if eval(varname):
            result.setSucceeded("Exercise solved.")
        else:
            result.setFailed("Exercise not solved.")

In [ ]:
@jagl.testcase("2", desc="Compiling program", deps=["1"])
def testcase_1_2(result, suite, case):
    if os.path.isfile(executable_name):
        result.setSucceeded("The program could be compiled successfully.")
        
    else:
        res = jagl.exec_bin("gcc", ["-o", executable_name] + source_files + cpp_flags)
        if res.state == jagl.ExecutionResult.SUCCESS:
            stdout = jagl.bytes_decode(res.stdout).strip()
            stderr = jagl.bytes_decode(res.stderr).strip()
            if res.exitCode == 0:
                if os.path.isfile(executable_name):
                    result.setSucceeded("The program could be compiled successfully.")
                else:
                    result.setFailed("gcc was executed successfully, but I couldn't find an executable.")
                if len(stderr) > 0:
                    result.setSucceeded(["However, there were warnings."] + stderr.split("\n"))
            else:
                result.setFailed(["The program could not be compiled:"] + stderr.split("\n"))
        elif res.state == jagl.ExecutionResult.NOTFOUND:
            result.setFailed("gcc could not be found. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.TIMEOUT:
            result.setFailed("Timeout while calling gcc. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.ERROR:
            result.setFailed(f'Unknown error during execution of gcc: {str(res.error)}. Please inform your course instructor.')

In [ ]:
@jagl.testcase("3", desc="Executing program", deps=["2"])
def testcase_1_3(result, suite, case):
    execresult = jagl.exec_bin("./" + executable_name, [], "22\n11\n2\nbla\n231\n232\n42\n23\n32\n23\n23\n34\n\n", timeout = 30)
    jagl.check_execution_result(result, execresult, checkExitCode = False)
    if execresult.exitCode is not None:
        if execresult.exitCode == -11:
            result.setFailed("Program caused a segmentation fault.")
        elif execresult.exitCode < 0:
            result.setFailed(f"Program was abnormally terminated (Signal: {-execresult.exitCode})")
        elif execresult.exitCode > 0:
            result.setFailed(f"Program returned an exit code other than 0 (Exit Code: {execresult.exitCode})")

In [ ]:
# new test suite
jagl.testsuite_begin("Exercise 2")

# delete all build artifacts
jagl.remove_paths(files=["quick_corr", "quick", "*.o"])

---
## 5.2 Makros und Konstanten (50%)

Im Ordner dieses Übungsblattes finden Sie die Quellcode-Datei "quick.c", welche eine Implementierung des QuickSort Sortieralgorithmus beinhaltet. In dieser Datei wurden einige Konstanten und Makros definiert, nur leider haben sich in den Code diverse Fehler eingeschlichen, welche eine erfolgreiche Kompilierung verhindern. Ihre Aufgabe ist es nun den Quellcode so zu ändern, dass eine erfolgreiche Kompilierung und Ausführung möglich ist. Es ist dabei nicht gestattet Konstanten und Makros zu entfernen, die bestehenden können aber abgeändert werden. Als Ausgabe des Programms wird das aufsteigend sortierte Array erwartet, wobei die einzelnen Elemente durch Tabulatoren getrennt werden sollen.

In [ ]:
%%bash

# Schreiben Sie in diese Zelle den Befehl der dafür nötig ist um Ihr Programm zu kompilieren.
# Beachten Sie dabei, dass der Name ihrer Quellcode-Datei "quick_corr.c" sein sollte.
# Beachten Sie dabei, dass der Name Ihres ausführbaren Programms "quick_corr" sein sollte.

gcc -Wall -Werror -o quick_corr quick_corr.c

In [ ]:
# Weisen Sie der untenstehenden Variable den Wert True zu, sobald Sie die Aufgabe erfolgreich erledigt haben!

# Datentyp: bool
exercise_5_2_solved = True

In [ ]:
executable_name = "quick_corr"
source_files = ["quick_corr.c"]
cpp_flags = []

In [ ]:
@jagl.testcase("1", desc="Exercise Solved")
def testcase_2_1(result, suite, case):
    varname = "exercise_5_2_solved"
    result.setSucceeded()
    jagl.check_variable_exists_and_has_type(result, globals(), varname, bool)
    if result.isSucceeded():
        if eval(varname):
            result.setSucceeded("Exercise solved.")
        else:
            result.setFailed("Exercise not solved.")

In [ ]:
@jagl.testcase("2", desc="Compiling program", deps=["1"])
def testcase_2_2(result, suite, case):
    if os.path.isfile(executable_name):
        result.setSucceeded("The program could be compiled successfully.")
        
    else:
        res = jagl.exec_bin("gcc", ["-o", executable_name] + source_files + cpp_flags)
        if res.state == jagl.ExecutionResult.SUCCESS:
            stdout = jagl.bytes_decode(res.stdout).strip()
            stderr = jagl.bytes_decode(res.stderr).strip()
            if res.exitCode == 0:
                if os.path.isfile(executable_name):
                    result.setSucceeded("The program could be compiled successfully.")
                else:
                    result.setFailed("gcc was executed successfully, but I couldn't find an executable.")
                if len(stderr) > 0:
                    result.setSucceeded(["However, there were warnings."] + stderr.split("\n"))
            else:
                result.setFailed(["The program could not be compiled:"] + stderr.split("\n"))
        elif res.state == jagl.ExecutionResult.NOTFOUND:
            result.setFailed("gcc could not be found. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.TIMEOUT:
            result.setFailed("Timeout while calling gcc. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.ERROR:
            result.setFailed(f'Unknown error during execution of gcc: {str(res.error)}. Please inform your course instructor.')

In [ ]:
@jagl.testcase("3", desc="Executing program", deps=["2"])
def testcase_2_3(result, suite, case):
    execresult = jagl.exec_bin("./" + executable_name, timeout = 30)
    jagl.check_execution_result(result, execresult, checkExitCode = False)
    if execresult.exitCode is not None:
        if execresult.exitCode == -11:
            result.setFailed("Program caused a segmentation fault.")
        elif execresult.exitCode < 0:
            result.setFailed(f"Program was abnormally terminated (Signal: {-execresult.exitCode})")
        elif execresult.exitCode > 0:
            result.setFailed(f"Program returned an exit code other than 0 (Exit Code: {execresult.exitCode})")
